# NiceCX v2.0 networkx conversion, remove 0 degree nodes upload

In this tutorial we will be downloading a network from NDEx, converting that network to networkx `Graph()` 
object via the `to_networkx()` method, removing 0 degree nodes, and uploading the new network
back to NDEx. 

# Import modules

Import needed modules. If there is an error with matplotlib run:

`conda install -c conda-forge matplotlib`

**NOTE:** For better support for Anaconda environments, exit Jupyter and run:
          `conda install -c anaconda nb_conda`

In [ ]:
import matplotlib
# this matplotlib inline enables the matplotlib plots to be displayed in the Jupyter notebook
%matplotlib inline

import matplotlib.pyplot as plt
import ndex2
import networkx

# used to prompt user for NDEx password in this notebook
import getpass


# Download network from NDEx

Using the `create_nice_cx_from_user()` the following line of code downloads the network from NDEx and creates a `NiceCXNetwork` object named `nice_cx_network`. 

For help on function names try running `help(nice_cx_network)`

In [ ]:
# EXAMPLE NETWORK FROM NDEx CIViC Variant-Drug Associations
# Viewable in a browser here: http://ndexbio.org/#/network/b9705e4f-57ef-11e9-9f06-0ac135e8bacf

nice_cx_network = ndex2.create_nice_cx_from_server('public.ndexbio.org', 
                                                   uuid='b9705e4f-57ef-11e9-9f06-0ac135e8bacf')




# Convert and print information about network

This fragment of code uses `to_networkx()` to convert the network to a networkx `Graph()` object. 

**NOTE:** The `to_networkx()` method creates a networkx `Graph()` object which does NOT allow
          multiple edges so in the conversion extra edges are lost.

In [ ]:

g = nice_cx_network.to_networkx()


print('NiceCXNetwork number of nodes: ' + str(len(nice_cx_network.get_nodes())) + 
      ' vs Networkx: ' + str(len(list(g))))
print('NiceCXNetwork number of edges: ' + str(len(nice_cx_network.get_edges())) +
      ' vs Networkx: ' + str(g.number_of_edges()))

# Plot network in notebook

If the following fails, the issue might be due to installation of older version of networkx (1.11) 

To fix this, exit jupyter and run the following in the terminal:

`pip install networkx --upgrade`

Then restart jupyter notebook (ie `jupyter notebook`) 

In [ ]:
networkx.draw(g)

# Get list of zero degree nodes

Using the networkx method `isolates()` get a list of all zero degree nodes and store them in the variable `isolatelist`

In [ ]:
isolatelist = list(networkx.isolates(g))

print('Number of zero degree nodes: ' + str(len(isolatelist)))


# Remove the zero degree nodes by iterating through list

Iterate through list and remove the zero degree nodes.

In [ ]:
print('Number of nodes before removal: ' + str(len(list(g))))

g.remove_nodes_from(isolatelist)

print('Number of nodes after removal: ' + str(len(list(g))))

# Convert networkx object back to NDEx NiceCXNetwork object

The following code converts the networkx graph object `g` back to a 
`NiceCXNetwork` object using `create_nice_cx_from_networkx()`

Additional calls below set the name of the network as well as the description. 

Finally for debugging the number of nodes and edges is output

In [ ]:
# takes a networkx object, g in this case, and creates NiceCXNetwork object (netty)
newnetwork = ndex2.create_nice_cx_from_networkx(g)

# sets the name of the network
newnetwork.set_name(nice_cx_network.get_name() + ' 0 degree nodes removed')

# remove the coordinates for now
newnetwork.remove_opaque_aspect('cartesianLayout')

# sets the description for the network
newnetwork.set_network_attribute('description', values='my network with 0 degree nodes removed')

print('NiceCXNetwork from networkx number of nodes: ' + str(len(newnetwork.get_nodes())) + 
      ' vs Networkx: ' + str(len(g)))
print('NiceCXNetwork from networkx number of edges: ' + str(len(newnetwork.get_edges())) +
      ' vs Networkx: ' + str(g.number_of_edges()))

# Prompt for NDEx username 

Be sure to hit enter in the field to set the value!!

In [ ]:
ndexuser = getpass.getpass()

# Prompt user for NDEx password 

Be sure to hit enter in the field to set the value!!

In [ ]:
ndexpassword = getpass.getpass()

# Upload network to NDEx

In [ ]:
res = newnetwork.upload_to('public.ndexbio.org', ndexuser, ndexpassword)

print('URL returned by upload_to call: ' + res)
justuuid = res.split('/')[-1]

print('Network UUID: ' + justuuid)

# Apply the style from the original network

When converting to/from networkx Graph() the style on the original network is lost. The `apply_template()` method
retreives the style from a network in NDEx and applies it to the current network. 

In [ ]:
newnetwork.apply_template('public.ndexbio.org', 'b9705e4f-57ef-11e9-9f06-0ac135e8bacf')

# Update existing network on NDEx

Rather then uploading a new network the following `update_to()` method replaces the existing network. This method requires the Network UUID to be set as the first argument.

In [ ]:
newnetwork.update_to(justuuid, 'public.ndexbio.org', ndexuser, ndexpassword)

In [ ]:
print('Tutorial complete. Have a nice day!')